# Análise Exploratória de Dados (EDA) - SRAG

Este notebook realiza a análise exploratória dos dados de Síndrome Respiratória Aguda Grave (SRAG) armazenados no banco de dados SQLite.

### Objetivos:
1. Verificar estrutura e qualidade dos dados.
2. Analisar valores ausentes.
3. Explorar distribuições das variáveis principais.
4. Analisar a variável alvo (Evolução/Óbito).

In [ ]:
import pandas as pd
import sqlite3
import matplotlib.pyplot as plt
import seaborn as sns

# Configurar estilo dos gráficos
sns.set_theme(style="whitegrid")
plt.rcParams['figure.figsize'] = (12, 6)

## 1. Carregamento dos Dados

In [ ]:
db_path = '../../data/database/srag.db'
conn = sqlite3.connect(db_path)

# Carregar dados da tabela principal (assumindo nome 'srag_records' ou similar - verificar tabelas)
query_tables = "SELECT name FROM sqlite_master WHERE type='table';"
tables = pd.read_sql(query_tables, conn)
print("Tabelas no banco:", tables)

# Carregar dados (ajustar nome da tabela conforme output acima)
table_name = tables.iloc[0, 0] # Pega a primeira tabela
df = pd.read_sql(f"SELECT * FROM {table_name}", conn)
conn.close()

df.head()

## 2. Visão Geral e Estrutura

In [ ]:
print(f"Linhas: {df.shape[0]}, Colunas: {df.shape[1]}")
df.info()

## 3. Análise de Valores Ausentes

In [ ]:
missing = df.isnull().sum()
missing = missing[missing > 0].sort_values(ascending=False)
missing_pct = (missing / df.shape[0]) * 100

plt.figure(figsize=(10, 8))
sns.barplot(x=missing_pct.values, y=missing_pct.index)
plt.title("Porcentagem de Valores Ausentes por Coluna")
plt.xlabel("%")
plt.show()

## 4. Análise da Variável Alvo (EVOLUCAO)
Código da evolução do caso:
1-Cura, 2-Óbito, 3-Óbito por outras causas, 9-Ignorado

In [ ]:
if 'EVOLUCAO' in df.columns:
    plt.figure(figsize=(8, 5))
    sns.countplot(data=df, x='EVOLUCAO')
    plt.title("Distribuição da Evolução dos Casos")
    plt.show()
    
    # Taxa de Mortalidade (Óbitos / Total com Evolução conhecida)
    evolucao_conhecida = df[df['EVOLUCAO'].isin([1, 2, 3])]
    obitos = evolucao_conhecida[evolucao_conhecida['EVOLUCAO'].isin([2, 3])].shape[0]
    total = evolucao_conhecida.shape[0]
    if total > 0:
        mortalidade = (obitos / total) * 100
        print(f"Taxa de Mortalidade (considerando apenas casos encerrados): {mortalidade:.2f}%")
else:
    print("Coluna EVOLUCAO não encontrada.")

## 5. Análise Temporal (DT_NOTIFIC)

In [ ]:
if 'DT_NOTIFIC' in df.columns:
    df['DT_NOTIFIC'] = pd.to_datetime(df['DT_NOTIFIC'], errors='coerce')
    cases_by_date = df.groupby('DT_NOTIFIC').size()
    
    plt.figure(figsize=(14, 6))
    cases_by_date.plot()
    plt.title("Casos de SRAG por Data de Notificação")
    plt.ylabel("Número de Casos")
    plt.xlabel("Data")
    plt.show()
else:
    print("Coluna DT_NOTIFIC não encontrada.")

## 6. Distribuição por Idade (NU_IDADE_N)

In [ ]:
if 'NU_IDADE_N' in df.columns:
    plt.figure(figsize=(10, 6))
    sns.histplot(df['NU_IDADE_N'], bins=30, kde=True)
    plt.title("Distribuição de Idade dos Pacientes")
    plt.xlabel("Idade")
    plt.show()